In [6]:
#Import relevant APIs
import numpy as np
import pandas as pd
import tweepy as tw
import yfinance as yf
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import date
from datetime import timedelta
from time import process_time

Setting up Twitter Authentication:

In [2]:
from tweepy import OAuthHandler
from tweepy import API

apiKey = 'sUY1UAhfmauoDAqodxPJCmE1d'
apiSecret = 'nRrYuBn8X1MauIFfyznAXdwdLsTcVReytlIUpsfjiI2GncodX0'
accessToken = '3295318197-LMzVBdXwZPbIgJU8Fd0ksjT8mEOES2HRYC2J2ha'
accessSecret = 'RgqkfaXowXGk1JaKs0jTOIvGVjZbGnL32bsPk4DuzohBf'

auth = OAuthHandler(apiKey, apiSecret)
auth.set_access_token(accessToken, accessSecret)
api = API(auth, wait_on_rate_limit = True)

Storing methods: Getting Ticker data and storing in Pandas DataFrame, then dict with only percent change:

In [7]:
tickers = ['TSLA','AAPL', 'FB', 'ZM', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMD', 'AMGN', 'AMZN',
          'ANSS', 'ASML', 'ATVI', 'AVGO', 'BIDU', 'BIIB', 'BKNG', 'BMRN', 'CDNS', 'CDW', 'CERN', 'CHKP', 'CHTR', 'CMCSA',
          'COST', 'CPRT', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'DLTR', 'DOCU', 'DXCM', 'EA', 'EBAY', 'EXC', 'EXPE',
          'FAST', 'FISV', 'FOX', 'GILD', 'GOOGL', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD', 'KDP', 'KHC',
          'KLAC', 'LBTYA', 'LRCX', 'LULU', 'MAR', 'MCHP', 'MDLZ', 'MELI', 'MNST', 'MRNA', 'MSFT', 'MU', 'MXIM',
          'NFLX', 'NTES', 'NVDA', 'NXPI', 'ORLY', 'PAYX', 'PCAR', 'PDD', 'PEP', 'PYPL', 'QCOM', 'REGN', 'ROST', 'SBUX',
          'SGEN', 'SIRI', 'SNPS', 'SPLK', 'SWKS', 'TCOM', 'TMUS', 'TTWO', 'TXN', 'ULTA', 'VRSK', 'VRSN', 'VRTX',
          'WBA', 'WDAY', 'XEL', 'XLNX']
perChange = {}
perChangeSelected = {}
selected_tickers = []

def storePercentChange():
    global end_date
    global days_prior
    global start_date
    end_date = date.today()
    days_prior = input("Enter the number of days to retrieve selected ticker & tweet data (max 7 days, min 2 days):  ")
    start_date = end_date - timedelta(days=int(days_prior))
    weekno = start_date.weekday()
    if weekno < 5 and int(days_prior) <= 7 and int(days_prior) >= 2:
        stock_data = yf.download(tickers, start=start_date, end=end_date, progress=False, group_by='ticker')
        for tick in tickers:
            percent_change = (((stock_data[tick]['Close'])[-1] - (stock_data[tick]['Close'])[str(start_date)])/
                (stock_data[tick]['Close'])[str(start_date)])*100
            perChange[tick] = round(abs(percent_change), 2)
    elif weekno >=5:
        sys.exit("Input leads to a date on the weekend. Exiting...")
    elif int(days_prior) > 7 or int(days_prior) < 2:
        sys.exit("Input does not match constraints. Exiting...")

Sorting Algorithms for all tickers in NQ100:

Selecting Top 5 & Bottom 5 Tickers in terms of % change and storing in list and dict:

Searching Twitter tweets using Tweepy:

In [4]:
tweetsDict = {}

def searchTweets():
    public_tweets = []
    count = 0
    counter = 0
    maxID = 0
    for tick in selected_tickers:
        while counter < 10:
            if maxID > 0:
                public_tweets = api.search(q = '$'+tick+' -filter:retweets', count = 100, 
                                           since = start_date, until = end_date, max_id = maxID)
            else:
                public_tweets = api.search(q = '$'+tick+' -filter:retweets', count = 100, 
                                           since = start_date, until = end_date)
            count += len(public_tweets)
            maxID = public_tweets[-1].id
            counter += 1
        tweetsDict[tick] = count 
        counter = 0
        count = 0
        maxID = 0

Merging two dicts on Ticker axis, storing in DF and Plotting:

In [5]:
def mergeAndPlot():
    corr_list = [perChange, tweetsDict]
    correlation = {}

    for tick in perChange.keys():
        correlation[tick] = tuple(correlation[tick] for correlation in corr_list)
    
    initial_table = pd.DataFrame(correlation)
    final_table = initial_table.transpose()
    final_table.columns = ['% Change', 'Number of Tweets']
    print(final_table)
    final_table.plot(kind = 'scatter', x='Number of Tweets', y='% Change', title = 'Tweets vs. % Change')
    

In [8]:
storePercentChange()
print(perChange)
#searchTweets()
#mergeAndPlot()

Enter the number of days to retrieve selected ticker & tweet data (max 7 days, min 2 days):   5


{'TSLA': 5.54, 'AAPL': 2.69, 'FB': 0.99, 'ZM': 14.29, 'ADBE': 1.57, 'ADI': 3.52, 'ADP': 0.52, 'ADSK': 0.8, 'ALGN': 8.4, 'ALXN': 1.31, 'AMAT': 7.71, 'AMD': 1.49, 'AMGN': 3.23, 'AMZN': 0.17, 'ANSS': 1.6, 'ASML': 6.09, 'ATVI': 1.76, 'AVGO': 2.52, 'BIDU': 3.65, 'BIIB': 2.37, 'BKNG': 4.76, 'BMRN': 1.16, 'CDNS': 1.75, 'CDW': 2.31, 'CERN': 0.73, 'CHKP': 2.16, 'CHTR': 2.88, 'CMCSA': 2.89, 'COST': 4.68, 'CPRT': 2.91, 'CSCO': 3.16, 'CSX': 2.0, 'CTAS': 2.3, 'CTSH': 3.75, 'CTXS': 5.33, 'DLTR': 2.99, 'DOCU': 6.73, 'DXCM': 8.87, 'EA': 0.74, 'EBAY': 1.07, 'EXC': 0.34, 'EXPE': 3.29, 'FAST': 2.83, 'FISV': 1.69, 'FOX': 3.08, 'GILD': 1.47, 'GOOGL': 3.95, 'IDXX': 0.83, 'ILMN': 8.87, 'INCY': 1.66, 'INTC': 7.53, 'INTU': 4.71, 'ISRG': 7.48, 'JD': 0.19, 'KDP': 0.26, 'KHC': 3.58, 'KLAC': 4.66, 'LBTYA': 11.1, 'LRCX': 10.45, 'LULU': 1.85, 'MAR': 6.89, 'MCHP': 7.33, 'MDLZ': 2.75, 'MELI': 0.26, 'MNST': 3.14, 'MRNA': 0.14, 'MSFT': 0.14, 'MU': 14.43, 'MXIM': 4.06, 'NFLX': 1.55, 'NTES': 3.31, 'NVDA': 1.17, 'NXPI': 3.